In [1]:
# check gpu
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-04-22 23:26:22.263985: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 23:26:22.937797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2024-04-22 23:26:23.671817: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 23:26:23.698184: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 23:26:23.698439: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
# load data and split into train and test sets
from load_data import sim_arr
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(sim_arr, test_size=0.2, random_state=42)

In [3]:
from autoencoder import Autoencoder
from hyper_optim import Hyper

# hyperparameter optimization
hyper = Hyper(Autoencoder, 3, train_set, test_set)
study = hyper.optimize(direction="minimize",
                       study_name="autoencoder",
                       storage="autoencoder.db",
                       n_trials=1)

# Get the best hyperparameters
best_params = study.best_params
for key, value in best_params.items():
    if key == 'activation':
        print(key, ":", value.__name__)
    else:
        print(key, ":", value)
print("Best hyperparameters:", best_params)
print("Best value:", study.best_value)

[I 2024-04-22 23:26:37,155] Using an existing study with name 'autoencoder' instead of creating a new one.


  0%|          | 0/1 [00:00<?, ?it/s]

/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2024-04-22 23:26:37.294991: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 23:26:37.295269: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 23:26:37.295455: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA

Epoch 1/10
[W 2024-04-22 23:26:37,651] Trial 6 failed with parameters: {'num_layers': 2, 'filters_0': 5, 'kernel_size0': 17, 'kernel_size1': 18, 'strides_0': 2, 'strides_1': 1, 'activation_0': 'relu', 'activation_1': 'relu', 'initializer_0': 'lecun_normal', 'initializer_1': 'glorot_normal'} because of the following error: ValueError('Exception encountered when calling MaxPooling2D.call().\n\n\x1bNegative dimension size caused by subtracting 5 from 2 for \'{{node autoencoder_1/encoder_1/max_pooling2d_2_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 5, 5, 1], padding="VALID", strides=[1, 5, 5, 1]](autoencoder_1/encoder_1/conv2d_1_2/Relu)\' with input shapes: [?,2,2,3].\x1b\n\nArguments received by MaxPooling2D.call():\n  • inputs=tf.Tensor(shape=(None, 2, 2, 3), dtype=float32)').
Traceback (most recent call last):
  File "/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, 

ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 5 from 2 for '{{node autoencoder_1/encoder_1/max_pooling2d_2_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 5, 5, 1], padding="VALID", strides=[1, 5, 5, 1]](autoencoder_1/encoder_1/conv2d_1_2/Relu)' with input shapes: [?,2,2,3].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(None, 2, 2, 3), dtype=float32)

In [ ]:
# train the model with the best hyperparameters
hyperparams = Hyper.search_space(study.best_trial)
model = Autoencoder(*hyperparams, output_dim=3)

model.fit(train_set, test_set, epochs=10, batch_size=32)
model.evaluate(test_set, test_set)

In [ ]:
# summarize the model
model.get_layer('encoder').summary()
model.get_layer('decoder').summary()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i = np.random.randint(0, len(test_set))
in_sim = test_set[i: i + 1]
out_sim = model(in_sim)
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(in_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")
ax[1].imshow(out_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")

In [ ]:
from tensorflow.keras.layers import Flatten as Flatten

feature_arr = model.feature_extractor(sim_arr)

In [ ]:
plt.scatter(feature_arr[:, 0],
            feature_arr[:, 1],
            feature_arr[:, 2])